In [1]:
import os
import cv2
import mediapipe as mp
import pprint
import numpy as np
import pandas as pd
import torch

robienie dataset

In [ ]:

# merging data
from pathlib import Path
import shutil

dataset_dir = "Sign-Language-Digits-Dataset-master\Dataset"
dataset_dir_2 = "data_2"

dataset_dirs = [dataset_dir , dataset_dir_2]

out_dir = Path("merged_dataset")
out_dir.mkdir(exist_ok=True)

for d in dataset_dirs:
    for cls in os.listdir(d):
        src = Path(d) / cls
        if not src.is_dir(): continue
        dst = out_dir / cls
        dst.mkdir(exist_ok=True)
        for f in os.listdir(src):
            shutil.copy(src / f, dst / f)




In [5]:

mp_hands =  mp.solutions.hands
hands_model  = mp_hands.Hands(static_image_mode=True, max_num_hands=1 , min_detection_confidence=0.1)


data = []
errors = []

count = 0
merged_data_dir = "merged_dataset"
for class_name in os.listdir(merged_data_dir):
    class_dir = os.path.join(merged_data_dir , class_name)


    for image_file in os.listdir(class_dir):
        count +=1
        image_path = os.path.join(class_dir , image_file)
        image  =cv2.imread(image_path)
        image =  cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image , (128 , 128))
        results = hands_model.process(image)
        
        if results.multi_hand_landmarks:
            hand_landmarks = results.multi_hand_landmarks[0]
            points = torch.tensor([[p.x , p.y , p.z] for p in hand_landmarks.landmark])
            data.append((points ,class_name))
        else:
            errors.append(image_path)
        
        if count%1000 == 0:
            print(count , len(errors))

   
torch.save(data , "tensordata" )        


1000 21
2000 52
3000 95
4000 98
5000 102
6000 111
7000 111
8000 111
9000 137
10000 145
11000 152
12000 166
13000 166
14000 166
15000 166
16000 166
17000 166
18000 166
19000 166
20000 166
21000 166
22000 166
23000 166
24000 166
25000 166
26000 166
27000 166
28000 166
29000 166
30000 168
31000 187
32000 432
33000 524
34000 534
35000 552
36000 577
37000 577
38000 578
39000 578
40000 578
41000 578
42000 578
43000 578
44000 578


In [ ]:
image_path = "merged_dataset/S/0.JPG"
image = cv2.imread(image_path)
print("Shape obrazu:", image.shape)  

Shape obrazu: (128, 128, 3)


In [ ]:
#moj komputer nie daje rady z przetwarzaniem większej ilości niż 

from concurrent.futures import ThreadPoolExecutor


merged_data_dir = "merged_dataset"


def process_image(class_name, img_file):

    mp_hands = mp.solutions.hands
    hands_model = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.1)    
    
    img_path = os.path.join(merged_data_dir, class_name, img_file)
    
    image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    
    results = hands_model.process(image)
    
    if results.multi_hand_landmarks:
        
        pts = torch.tensor([[p.x, p.y, p.z] for p in results.multi_hand_landmarks[0].landmark])
        
        return (pts, int(class_name))
    return None


tasks = [(class_name, img_file) 
         
         for class_name in os.listdir(merged_data_dir)
         
         for img_file in os.listdir(os.path.join(merged_data_dir, class_name))]

data = []

count = 0
with ThreadPoolExecutor(max_workers=2) as executor:
    
    for res in executor.map(lambda t: process_image(*t), tasks):
        count +=1
        if res is not None:
            
            data.append(res)
        if count % 1000 ==0:
            print(f"{count} ilość obrazów {len(data)} ")

torch.save(data, "hands_tensor_data_multithread.pt")
print(f"Gotowe! Liczba próbek: {len(data)}")

: 

In [ ]:
total = sum(len(files) for _, _, files in os.walk(merged_data_dir))
print("Liczba wszystkich plików:", total)

Liczba wszystkich plików: 44062


: 

szacowanie ile zajmie przejście przez dataset

In [ ]:
import time
mp_hands = mp.solutions.hands
hands_model = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.1)

image_path = r"merged_dataset\0\IMG_1128.JPG"  

image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
start = time.time()

results = hands_model.process(image)

if results.multi_hand_landmarks:
    hand_landmarks = results.multi_hand_landmarks[0]
    points = torch.tensor([[p.x, p.y, p.z] for p in hand_landmarks.landmark])
    print("Tensor shape:", points.shape)

end = time.time()
print("Czas przetwarzania:", end - start, "sekund")

Tensor shape: torch.Size([21, 3])
Czas przetwarzania: 0.06308627128601074 sekund


In [3]:
os.cpu_count()

6

In [2]:
tensor_data =  torch.load("tensordata")

In [3]:
labels = os.listdir("merged_dataset")
label_map = {k : v  for v , k in enumerate(labels)}

X = torch.stack([t[0] for t in tensor_data])
Y = torch.stack([torch.tensor(label_map[t[1]]) for t in tensor_data])
X.shape

X = X.unsqueeze(1)

In [4]:
X.shape

torch.Size([43484, 1, 21, 3])

In [8]:
from torch.utils.data import TensorDataset ,DataLoader ,random_split


dataset = TensorDataset(X , Y)

train_len = int(0.8 * len(dataset))
valid_len= len(dataset) - train_len

train_dataset , valid_dataset = random_split(dataset , [train_len , valid_len])

traning_loader = DataLoader( train_dataset , batch_size= 32, shuffle=True)
valid_loader = DataLoader(valid_dataset , batch_size=32 , shuffle=False)


In [5]:
label_map

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 'A': 10,
 'B': 11,
 'C': 12,
 'D': 13,
 'E': 14,
 'F': 15,
 'G': 16,
 'H': 17,
 'I': 18,
 'J': 19,
 'K': 20,
 'L': 21,
 'M': 22,
 'N': 23,
 'O': 24,
 'P': 25,
 'Q': 26,
 'R': 27,
 'S': 28,
 'T': 29,
 'U': 30,
 'V': 31,
 'W': 32,
 'X': 33,
 'Y': 34,
 'Z': 35}

In [ ]:
import torch.nn as nn

class Sing_language_model(nn.Module):
    def __init__(self , input_size , hidden_size ):
        pass
        

    def forward(self):
        pass

    def detect():
        pass

: 